In [1]:
import geopandas as gpd
import osmium as osm
import shapely as shp

ModuleNotFoundError: No module named 'osmium'

In [ ]:
    def node(self, n):    
        self.coords = shp.Point((n.location.x, n.location.y))
        self.tag_inventory(n, "node")
        
        del n

    def way(self, w):
        test = []
        for i in range(len(w.nodes)):
            test.append((w.nodes[i].location.x, w.nodes[i].location.y))
        
        # this only works if we have more than 4 coordinates if not then throw this data entry away
        if (4 <= len(test)):
            self.coords = shp.Polygon(test.copy())
            self.tag_inventory(w, "way")
        
        # if you dont do this the libary sometimes, throws erros, maybe only jupyter issues needs furtehr testing
        del w
        del test

In [ ]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.coords = 0
        self.tag_list = []
        
    def tag_inventory(self, elem, elem_type):
        self.tag_list = []
        for tag in elem.tags:
            self.tag_list.append((tag.k, tag.v))
        
        if (elem_type == "way"):
            self.osm_data.append([elem_type, 
                                   self.coords,
                                   len(self.tag_list),
                                   self.tag_list])
        elif (elem_type == "way"):
            self.osm_data.append([elem_type, 
                                   0,
                                   len(self.tag_list),
                                   self.tag_list])
        else:
            self.osm_data.append([elem_type, 
                                  self.coords,
                                  len(self.tag_list),
                                  self.tag_list])
            
    
    def node(self, n):    
        self.coords = shp.Point((n.location.x, n.location.y))
        self.tag_inventory(n, "node")
        
        del n

    def way(self, w):
        test = []
        for i in range(len(w.nodes)):
            test.append((w.nodes[i].location.x, w.nodes[i].location.y))
        
        # this only works if we have more than 4 coordinates if not then throw this data entry away
        if (4 <= len(test)):
            self.coords = shp.Polygon(test.copy())
            self.tag_inventory(w, "way")
        
        # if you dont do this the libary sometimes, throws erros, maybe only jupyter issues needs furtehr testing
        del w
        del test

    # its really hard to get the location out of relations
    def relation(self, r):   
        self.tag_inventory(r, "relation")
        
        del r

In [ ]:
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("../../data/andorra-latest.osm.pbf", locations=True)

In [ ]:
# transform the list into a pandas DataFrame
data_colnames = ['osm_type','location', 'ntags', 'tags']
df_osm = gpd.GeoDataFrame(osmhandler.osm_data, columns=data_colnames)
# df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [ ]:
df_osm.head(20)